In [14]:
import os
import sys
import json
from copy import deepcopy
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

In [15]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [16]:
!ls data_files/store_model_scores/sorted

test_set_q_candidate_model_scores_0.json
test_set_q_candidate_model_scores_1.json
test_set_q_candidate_model_scores_2.json
test_set_q_candidate_model_scores_3.json


#### Load the sorted scores thing (assume zero-based numbeering)

We need:
   1) For each question pair, 
       1st ranked question and its score
       best ranked duplicate and its score
       worst ranked duplicate and score
       did it happen etc

In [17]:
INPUT_FILE_PATH="data_files/store_model_scores/sorted_pruned/"

In [18]:
file_idx=0

with open(INPUT_FILE_PATH+f"test_set_q_candidate_model_scores_{file_idx}.json",'r') as fd:
    df=json.load(fd)

df['440482']

In [19]:
THRESHOLD=20

In [20]:
details=dict()

 ### Showing reduction in search space

In [21]:
curr_file_id=0
with open(INPUT_FILE_PATH+f"test_set_q_candidate_model_scores_{curr_file_id}.json",'r') as fd:
        df=json.load(fd)

In [22]:
def fetch_stats(q_obj,model_idx ):
    # first sort this by model id
    sorted_list=sorted(q_obj['scores'], key=lambda x:x['rank'][model_idx], reverse=False)
    best_ans=None
    ans=dict()
    ############################
    seeked_idx=0
    ans['model_id']=model_idx
    ans['best_ans']=[sorted_list[seeked_idx]['candidate_qid'],sorted_list[seeked_idx]['rank'][model_idx],\
                    sorted_list[seeked_idx]['cm_s'][model_idx]]
    #############################
    sorted_list=list(filter(lambda x:x['candidate_qid'] in q_obj['expected_questions'], sorted_list))
    ###########################
    seeked_idx=0
    ans['best_dupe']=[sorted_list[seeked_idx]['candidate_qid'],sorted_list[seeked_idx]['rank'][model_idx],\
                    sorted_list[seeked_idx]['cm_s'][model_idx]]
    ######################
    seeked_idx=-1
    ans['worst_dupe']=[sorted_list[seeked_idx]['candidate_qid'],sorted_list[seeked_idx]['rank'][model_idx],\
                    sorted_list[seeked_idx]['cm_s'][model_idx]]
    #########################
    ans['is_success']=ans['best_dupe'][1]<20
    return ans
    

In [23]:
df['440482']

{'expected_questions': [314242],
 'scores': [{'candidate_qid': 178899,
   'title_score': 0.5,
   'body_score': 0.4318335171,
   'tag_score': 0.7745966692,
   'topic_score': 0.3924670351,
   'jaccard_sim': 0.1315789474,
   'cm_s': [1.1731679119,
    1.0028568133,
    1.1609915191,
    0.2402135959,
    0.8347135172,
    0.8336745932],
   'rank': [0, 0, 0, 0, 2, 0]},
  {'candidate_qid': 321971,
   'title_score': 0.316227766,
   'body_score': 0.4575090109,
   'tag_score': 0.6708203932,
   'topic_score': 0.4285151397,
   'jaccard_sim': 0.225,
   'cm_s': [1.0098288298,
    0.9705202103,
    0.9329775718,
    0.2010317971,
    0.6934637121,
    0.8318721985],
   'rank': [6, 1, 10, 5, 14, 1]},
  {'candidate_qid': 393886,
   'title_score': 0.5,
   'body_score': 0.306924907,
   'tag_score': 0.632455532,
   'topic_score': 0.4950092071,
   'jaccard_sim': 0.1627906977,
   'cm_s': [1.0566810276,
    0.8411097537,
    1.1269779228,
    0.2055412186,
    0.7450408463,
    0.8049949758],
   'rank': [4

In [24]:
lb_file_idx=0
ub_file_idx=3
curr_smai_q_id=-1
x_c=[]
y_c=[]
# go through all files
for curr_file_id in range(lb_file_idx, ub_file_idx+1):
    with open(INPUT_FILE_PATH+f"test_set_q_candidate_model_scores_{curr_file_id}.json",'r') as fd:
        df=json.load(fd)
    print("file opened with id ", curr_file_id)

    # go through each question for whom we want a duplicate
    for curr_dup_q, curr_val in df.items():
        curr_smai_q_id+=1
        details[curr_dup_q]={"data":[]}
        for curr_model_id in range(6):
            details[curr_dup_q]['data'].append(fetch_stats(curr_val, curr_model_id))        

file opened with id  0
file opened with id  1
file opened with id  2
file opened with id  3


In [25]:
with open("./details.json",'w') as fd:
    json.dump(details, fd, indent=2)

In [26]:
details

{'440482': {'data': [{'model_id': 0,
    'best_ans': [178899, 0, 1.1731679119],
    'best_dupe': [314242, 36, 0.840617086],
    'worst_dupe': [314242, 36, 0.840617086],
    'is_success': False},
   {'model_id': 1,
    'best_ans': [178899, 0, 1.0028568133],
    'best_dupe': [314242, 13, 0.8354861126],
    'worst_dupe': [314242, 13, 0.8354861126],
    'is_success': True},
   {'model_id': 2,
    'best_ans': [178899, 0, 1.1609915191],
    'best_dupe': [314242, 50, 0.8032541708],
    'worst_dupe': [314242, 50, 0.8032541708],
    'is_success': False},
   {'model_id': 3,
    'best_ans': [178899, 0, 0.2402135959],
    'best_dupe': [314242, 47, 0.1542377309],
    'worst_dupe': [314242, 47, 0.1542377309],
    'is_success': False},
   {'model_id': 4,
    'best_ans': [372812, 0, 0.9250017896],
    'best_dupe': [314242, 86, 0.5510479637],
    'worst_dupe': [314242, 86, 0.5510479637],
    'is_success': False},
   {'model_id': 5,
    'best_ans': [178899, 0, 0.8336745932],
    'best_dupe': [314242, 3,

In [37]:
def fetch_pair(q1, q2):
    q1=str(q1)
    q2=q2
    
    for curr_file_id in range(lb_file_idx, ub_file_idx+1):
        with open(INPUT_FILE_PATH+f"test_set_q_candidate_model_scores_{curr_file_id}.json",'r') as fd:
            df=json.load(fd)
        #print("file opened with id ", curr_file_id)
        #print(q1 in df.keys())
        #print(df.keys())
        if q1 in df:
            for curr_item in df[q1]['scores']:
                if curr_item['candidate_qid']==q2:
                    return curr_item
        else:
            continue
    print("NOTHING")
    return None
    

In [40]:
fetch_pair(441529,34505 )

{'candidate_qid': 34505,
 'title_score': 0.5163977795,
 'body_score': 0.5077958716,
 'tag_score': 1.0,
 'topic_score': 0.7371697563,
 'jaccard_sim': 0.109375,
 'cm_s': [1.4287870041,
  1.3331125447,
  1.4849826576,
  0.2745620218,
  0.9447116744,
  0.9979114985],
 'rank': [0, 0, 0, 0, 0, 0]}

### Dig deeper

In [64]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)

ans=None
m1=0
m2=4
for curr_key, curr_val in details.items():
    if (curr_val['data'][m1]['is_success']==True) and (curr_val['data'][m2]['is_success']==False):
        print("Curr key is ", curr_key)
        pp.pprint(curr_val['data'][m1])
        print("#######")
        pp.pprint(curr_val['data'][m2])
        #print(curr_val)
        print("#############################3")
        

Curr key is  448230
{   'best_ans': [158436, 0, 1.4941181361],
    'best_dupe': [103785, 12, 1.0388063001],
    'is_success': True,
    'model_id': 0,
    'worst_dupe': [282805, 83, 0.8202169169]}
#######
{   'best_ans': [158436, 0, 1.1767667111],
    'best_dupe': [103785, 53, 0.6414518995],
    'is_success': False,
    'model_id': 4,
    'worst_dupe': [282805, 81, 0.6006463478]}
#############################3
Curr key is  454002
{   'best_ans': [400739, 0, 1.6070694316],
    'best_dupe': [58968, 14, 1.0655422123],
    'is_success': True,
    'model_id': 0,
    'worst_dupe': [58968, 14, 1.0655422123]}
#######
{   'best_ans': [400739, 0, 1.1606925792],
    'best_dupe': [58968, 27, 0.6927568541],
    'is_success': False,
    'model_id': 4,
    'worst_dupe': [58968, 27, 0.6927568541]}
#############################3
Curr key is  481300
{   'best_ans': [315000, 0, 1.5795524565],
    'best_dupe': [105049, 10, 1.3021457978],
    'is_success': True,
    'model_id': 0,
    'worst_dupe': [105049

In [65]:
fetch_pair(521687,43021 )

{'candidate_qid': 43021,
 'title_score': 0.5163977795,
 'body_score': 0.087038828,
 'tag_score': 1.0,
 'topic_score': 0.3578651984,
 'jaccard_sim': 0.125,
 'cm_s': [0.9515627404,
  0.8427124624,
  1.2953303787,
  0.1921652408,
  0.5701755838,
  0.7714453938],
 'rank': [3, 1, 2, 6, 28, 4]}

In [66]:
fetch_pair(521687,141108 )

{'candidate_qid': 141108,
 'title_score': 0.8164965809,
 'body_score': 0.25,
 'tag_score': 0.4082482905,
 'topic_score': 0.6427844673,
 'jaccard_sim': 0.1666666667,
 'cm_s': [1.2724677115,
  0.7001728015,
  1.3828377888,
  0.2415856929,
  0.9991253696,
  0.9023669697],
 'rank': [0, 12, 0, 0, 0, 0]}